## Matched Molecular Series  
### Introduction

This notebook implements matched molecular series as described in these papers. 

Wawer, Mathias, and Jürgen Bajorath. "Local structural changes, global data views: graphical substructure− activity relationship trailing." Journal of Medicinal Chemistry **54** (2011): 2944-2951. https://doi.org/10.1021/jm200026b

O’Boyle, Noel M., Jonas Boström, Roger A. Sayle, and Adrian Gill. "Using matched molecular series as a predictive tool to optimize biological activity." *Journal of Medicinal Chemistry*, **57** (2014): 2704-2713. [https://doi.org/10.1021/jm500022q](https://doi.org/10.1021/jm500022q)

### Setup
Check to see if we're running in Google Colab

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules

Install the necessary Python packages

In [2]:
if IN_COLAB:
    !pip install pandas requests rdkit useful_rdkit_utils mols2grid

Download the scaffold_finder library from GitHub.

In [3]:
if IN_COLAB:
    import requests
    lib_file = requests.get("https://raw.githubusercontent.com/PatWalters/practical_cheminformatics_tutorials/main/sar_analysis/scaffold_finder.py")
    ofs = open("scaffold_finder.py","w")
    print(lib_file.text,file=ofs)
    ofs.close()

In [4]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.rdMMPA import FragmentMol
from rdkit.Chem.rdDepictor import Compute2DCoords
from scaffold_finder import generate_fragments, cleanup_fragment
from operator import itemgetter
import mols2grid
from rdkit.Chem.TemplateAlign import AlignMolToTemplate2D

Useful utility functions

In [5]:
def sort_fragments(mol):
    """
    Transform a molecule with multiple fragments into a list of molecules that is sorted by number of atoms
    from largest to smallest
    """
    frag_list = list(Chem.GetMolFrags(mol, asMols=True))
    frag_num_atoms_list = [(x.GetNumAtoms(), x) for x in frag_list]
    frag_num_atoms_list.sort(key=itemgetter(0), reverse=True)
    return [x[1] for x in frag_num_atoms_list]

### Read Input
Read the input data

In [6]:
infile_url = "../data/CHEMBL208.smi"
chembl208_df = pd.read_csv(infile_url,names=["SMILES","Name","pIC50"])

Add an RDKit molecule to the dataframe

In [7]:
chembl208_df['mol'] = chembl208_df.SMILES.apply(Chem.MolFromSmiles)

### Generate Scaffold-Substituent Pairs
Iterate over the molecules in the dataframe and transform to scaffold, substituent pairs 

In [8]:
row_list = []
for smiles, name, pIC50, mol in chembl208_df.values:
    frag_list = FragmentMol(mol,maxCuts=1)
    for _,frag_mol in frag_list:
        pair_list = sort_fragments(frag_mol)
        row_list.append([smiles]+[Chem.MolToSmiles(x) for x in pair_list]+[name, pIC50])
row_df = pd.DataFrame(row_list,columns=["SMILES","Core","R_group","Name","pIC50"])
row_df

,SMILES,Core,R_group,Name,pIC50
0,O=c1[nH]c2ccc(-c3cccc(Cl)c3)cc2s1,O=c1[nH]c2ccc([*:1])cc2s1,Clc1cccc([*:1])c1,CHEMBL94053,6.90
1,O=c1[nH]c2ccc(-c3cccc(Cl)c3)cc2s1,O=c1[nH]c2ccc(-c3cccc([*:1])c3)cc2s1,Cl[*:1],CHEMBL94053,6.90
2,O=c1[nH]c2ccc(-c3cccc([N+](=O)[O-])c3)cc2n1Cc1...,O=c1[nH]c2ccc([*:1])cc2n1Cc1ccccc1,O=[N+]([O-])c1cccc([*:1])c1,CHEMBL327409,6.15
3,O=c1[nH]c2ccc(-c3cccc([N+](=O)[O-])c3)cc2n1Cc1...,O=c1[nH]c2ccc(-c3cccc([*:1])c3)cc2n1Cc1ccccc1,O=[N+]([O-])[*:1],CHEMBL327409,6.15
4,O=c1[nH]c2ccc(-c3cccc([N+](=O)[O-])c3)cc2n1Cc1...,O=c1[nH]c2ccc(-c3cccc([N+](=O)[O-])c3)cc2n1[*:1],c1ccc(C[*:1])cc1,CHEMBL327409,6.15
...,...,...,...,...,...
3083,Cc1c(N[C@@H]2CCC[C@@]2(C)O)ccc(C#N)c1Cl,Cc1c(N[*:1])ccc(C#N)c1Cl,C[C@@]1(O)CCC[C@H]1[*:1],CHEMBL3764185,6.06
3084,Cc1c(N[C@@H]2CCC[C@@]2(C)O)ccc(C#N)c1Cl,Cc1c(N[C@@H]2CCC[C@]2(O)[*:1])ccc(C#N)c1Cl,C[*:1],CHEMBL3764185,6.06
3085,Cc1c(N[C@@H]2CCC[C@@]2(C)O)ccc(C#N)c1Cl,Cc1c(N[C@@H]2CCC[C@]2(C)[*:1])ccc(C#N)c1Cl,O[*:1],CHEMBL3764185,6.06
3086,Cc1c(N[C@@H]2CCC[C@@]2(C)O)ccc(C#N)c1Cl,Cc1c(N[C@@H]2CCC[C@@]2(C)O)ccc([*:1])c1Cl,N#C[*:1],CHEMBL3764185,6.06


### View Scaffolds
Put the scaffolds into a dataframe

In [9]:
scaffold_df = row_df.Core.value_counts().to_frame().reset_index()
scaffold_df.columns = ["SMILES","Count"]
scaffold_df['mol'] = scaffold_df.SMILES.apply(Chem.MolFromSmiles)
_ = scaffold_df.mol.apply(Compute2DCoords)

Display the scaffolds

In [10]:
mols2grid.display(scaffold_df.head(20),subset=["img","Count"],mol_col="mol",
                  use_coords=True,prerender=True)

### View Matched Molecular Series  
Set **idx** to the index from the scaffold table above to display molecules from that series  
Set **clear_matches** to **False** to highlight the scaffold in the table below

In [11]:
idx = 4
clear_matches = False
scaffold_smi = scaffold_df.SMILES.values[idx]
scaffold_mol = scaffold_df.mol.values[idx]
scaffold_mol = cleanup_fragment(scaffold_mol)[0]
current_df = row_df.query("Core == @scaffold_smi").sort_values("pIC50",ascending=False).copy()
current_df['mol'] = current_df.SMILES.apply(Chem.MolFromSmiles)
for mol in current_df.mol.values:
    AlignMolToTemplate2D(mol, scaffold_mol)
    if clear_matches:
        mol.__sssAtoms = []
mols2grid.display(current_df,subset=["img","pIC50"],mol_col="mol",use_coords=True,prerender=True)